In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

from keras.callbacks import ModelCheckpoint,  CSVLogger
from keras.layers import Add, Dense, Input, LSTM
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
from matplotlib import pyplot as plt
from models import Generator2, create_training_model

In [ ]:
# transform char in integer
def numerical_encoding(text, char_dict):
    """ Text to list of chars, to np.array of numerical idx """
    chars_list = [ char for char in text ]
    chars_list = [ char_dict[char] for char in chars_list ]
    chars_list = np.array(chars_list)
    return chars_list


def get_text_matrix(sequence, len_input):
    
    # create empty matrix
    X = np.empty((len(sequence)-len_input, len_input))
    
    # fill each row/time window from input sequence
    for i in range(X.shape[0]):
        X[i,:] = sequence[i : i+len_input]
        
    return X

In [ ]:
# Settings

# Percent of samples to use for training, might be necessary if you're running out of memory
sample_size = 1

# The latent dimension of the LSTM
latent_dim = 2048

# Number of epochs to train for
epochs = 20

# path of the data
data_path = 'data/DivinaCommedia.csv'

name = 'all_data_test_2'
output_dir = Path('output_%s' % name)
output_dir.mkdir()

In [ ]:
df = pd.read_csv(str(data_path))
df = df.sample(frac=sample_size)


max_line_length = int(max([df['%s' % i].astype(str).str.len().quantile(.99) for i in range(3)]))

df = df[
    (df['0'].astype(str).str.len() <= max_line_length) & 
    (df['1'].astype(str).str.len() <= max_line_length) & 
    (df['2'].astype(str).str.len() <= max_line_length)
].copy()

# preprocessing data
# Pad the lines to the max line length with new lines
for i in range(3):
    # For input, duplicate the first character
    # TODO - Why?
    df['%s_in' % i] = (df[str(i)].str[0] + df[str(i)]).str.pad(max_line_length+2, 'right', '\n')
    
    # 
    #df['%s_out' % i] = df[str(i)].str.pad(max_line_len, 'right', '\n') + ('\n' if i == 2 else df[str(i+1)].str[0])
    
    # TODO - trying to add the next line's first character before the line breaks
    if i == 2: # If it's the last line
        df['%s_out' % i] = df[str(i)].str.pad(max_line_length+2, 'right', '\n')
    else: 
        # If it's the first or second line, add the first character of the next line to the end of this line.
        # This helps with training so that the next RNN has a better chance of getting the first character right.
        df['%s_out' % i] = (df[str(i)] + '\n' + df[str(i+1)].str[0]).str.pad(max_line_length+2, 'right', '\n')
    
max_line_length += 2

inputs = df[['0_in', '1_in', '2_in']].values


tokenizer = Tokenizer(filters='', char_level=True)
tokenizer.fit_on_texts(inputs.flatten())
n_tokens = len(tokenizer.word_counts) + 1


print(df)

# X is the input for each line in sequences of one-hot-encoded values
X = np_utils.to_categorical([
  tokenizer.texts_to_sequences(inputs[:,i]) for i in range(3)
  ], num_classes=n_tokens)

outputs = df[['0_out', '1_out', '2_out']].values

# Y is the output for each line in sequences of one-hot-encoded values
Y = np_utils.to_categorical([
    tokenizer.texts_to_sequences(outputs[:,i]) for i in range(3)
], num_classes=n_tokens)

# X_syllables is the count of syllables for each line
X_syllables = df[['0_syllables', '1_syllables', '2_syllables']].values




In [ ]:
training_model, lstm, lines, inputs, outputs = create_training_model(latent_dim, n_tokens)

filepath = str(output_dir / ("%s-{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5" % latent_dim))
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

csv_logger = CSVLogger(str(output_dir / 'training_log.csv'), append=True, separator=',')

callbacks_list = [checkpoint, csv_logger]

training_model.fit([
    X[0], X_syllables[:,0],
    X[1], X_syllables[:,1], 
    X[2], X_syllables[:,2]
], [Y[0], Y[1], Y[2]], batch_size=64, epochs=epochs, validation_split=.1, callbacks=callbacks_list)

In [ ]:
#Import Joblib

from sklearn.externals import joblib

# Load a copy of the training model from disk by creating a new model using the same parameters and then loading the weights.

output_dir = Path('output_all_data_test_2')

# Get the parameters used for creating the model
#latent_dim, n_tokens, max_line_length, tokenizer = joblib.load(output_dir / 'metadata.pkl')

# Create the new placeholder model
training_model, lstm, lines, inputs, outputs = create_training_model(latent_dim, n_tokens)

# Load the specified weights
training_model.load_weights(output_dir / '2048-20-1.32-4.88.hdf5')

In [ ]:
# Create a generator using the training model as the template

generator = Generator2(lstm, lines, tokenizer, n_tokens, max_line_length)

In [ ]:
for i in range(50):
    generator.generate_haiku([11, 11, 11])
    print()

In [ ]:
for i in range(50):
    generator.generate_haiku(temperature=.3)
    print()

In [ ]:
for i in range(50):
    generator.generate_haiku(temperature=.5)
    print()

In [ ]:
for i in range(50):
    generator.generate_haiku(temperature=.75)
    print()

In [ ]:
for i in range(50):
    generator.generate_haiku(temperature=1)
    print()